In [1]:
# Import Libraries

import requests
import pandas as pd
import time
import psycopg2 as ps
from config import API_KEY, host_name, dbname, port, username, password

In [2]:
# YouTube Channel ID

CHANNEL_ID = "UCsXVk37bltHxD1rDPwtNM8Q"

In [3]:
def get_video_details(video_id):
        
        #collecting view, like, comment counts
        url_video_stats = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+API_KEY
        response_video_stats = requests.get(url_video_stats).json()

        view_count = response_video_stats['items'][0]['statistics']['viewCount']
        like_count = response_video_stats['items'][0]['statistics']['likeCount']
        comment_count = response_video_stats['items'][0]['statistics']['commentCount']

        return view_count, like_count, comment_count

In [4]:
def get_videos(df):
    pageToken = "CDIQAA"
    while 1:
        url = "https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&channelId="+CHANNEL_ID+"&part=snippet,id&order=date&maxResults=100&"+pageToken

        response = requests.get(url).json()
        time.sleep(1)
    
        for video in response['items']:
            if video['id']['kind'] == "youtube#video":
                video_id = video['id']['videoId']
                video_title = video['snippet']['title']
                upload_date = video['snippet']['publishedAt']
                upload_date = str(upload_date).split("T")[0]
                view_count, like_count, comment_count = get_video_details(video_id)
        
                #save data in pandas df
                df = df.append({'video_id': video_id,
                                'video_title': video_title,
                                'upload_date': upload_date,
                                'view_count': view_count,
                                'like_count': like_count,
                                'comment_count' : comment_count},
                                ignore_index=True)
        try:
            if response['nextPageToken'] != None: #if none, it means it reached the last page and break out of it
                pageToken = "pageToken=" + response['nextPageToken']

        except:
            break
            
            
    return df
        

In [5]:
#main

#building the dataframe
df = pd.DataFrame(columns=["video_id", "video_title", "upload_date", "view_count", "like_count", "comment_count"])

df = get_videos(df)

In [6]:
df

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,xAUJYP8tnRE,Why We Should NOT Look For Aliens - The Dark F...,2021-12-14,7919526,476595,26123
1,XFqn3uy238E,...And We&#39;ll Do it Again,2021-12-07,8764985,594845,24183
2,F1Hq8eVOMHs,Is Meat Really that Bad?,2021-11-30,5862147,349009,41105
3,LmpuerlbJu0,You Are Immune Against Every Disease,2021-11-02,9256850,432016,22236
4,Nv4Nk4AAgk8,The Limited Edition Dinosaur Calendar – Now An...,2021-10-19,692609,36256,1660
...,...,...,...,...,...,...
148,F3QpgXBtDeo,How The Stock Exchange Works (For Dummies),2013-11-28,8001069,125406,8165
149,UuGrBhK2c7U,The Gulf Stream Explained,2013-10-11,4205519,57868,1927
150,Uti2niW2BRA,Fracking explained: opportunity or danger,2013-09-03,6966104,96311,8105
151,KsF_hdjWJjo,The Solar System -- our home in space,2013-08-22,5678582,78342,6052


In [ ]:
# df.to_csv('kurzdata.csv')

In [7]:
def connect_to_db(host_name, dbname, port, username, password):
    try:
        conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)

    except ps.OperationalError as e:
        raise e
    else:
        print('Connected!')
        return conn

def create_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS videos (
                    video_id VARCHAR(255) PRIMARY KEY,
                    video_title TEXT NOT NULL,
                    upload_date DATE NOT NULL DEFAULT CURRENT_DATE,
                    view_count INTEGER NOT NULL,
                    like_count INTEGER NOT NULL,
                    comment_count INTEGER NOT NULL
            )""")

    curr.execute(create_table_command)

def insert_into_table(curr, video_id, video_title, upload_date, view_count, like_count, comment_count):
    insert_into_videos = ("""INSERT INTO videos (video_id, video_title, upload_date,
                        view_count, like_count, comment_count)
    VALUES(%s,%s,%s,%s,%s,%s);""")
    row_to_insert = (video_id, video_title, upload_date, view_count, like_count, comment_count)
    curr.execute(insert_into_videos, row_to_insert)


def update_row(curr, video_id, video_title, view_count, like_count, comment_count):
    query = ("""UPDATE videos
            SET video_title = %s,
                view_count = %s,
                like_count = %s,
                comment_count = %s
            WHERE video_id = %s;""")
    vars_to_update = (video_title, view_count, like_count, comment_count, video_id)
    curr.execute(query, vars_to_update)


def check_if_video_exists(curr, video_id): 
    query = ("""SELECT video_id FROM VIDEOS WHERE video_id = %s""")

    curr.execute(query, (video_id,))
    return curr.fetchone() is not None



def truncate_table(curr):
    truncate_table = ("""TRUNCATE TABLE videos""")

    curr.execute(truncate_table)


def append_from_df_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_table(curr, row['video_id'], row['video_title'], row['upload_date'], row['view_count']
                          , row['like_count'], row['comment_count'])


def update_db(curr,df):
    tmp_df = pd.DataFrame(columns=['video_id', 'video_title', 'upload_date', 'view_count',
                                   'like_count', 'comment_count'])
    for i, row in df.iterrows():
        if check_if_video_exists(curr, row['video_id']): # If video already exists then we will update
            update_row(curr,row['video_id'],row['video_title'],row['view_count'],row['like_count'],row['comment_count'])
        else: # The video doesn't exists so we will add it to a temp df and append it using append_from_df_to_db
            tmp_df = tmp_df.append(row)

    return tmp_df

In [8]:
conn = connect_to_db(host_name, dbname, port, username, password)
curr = conn.cursor()

Connected!


In [9]:
create_table(curr)

In [10]:
new_vid_df = update_db(curr, df)

In [11]:
append_from_df_to_db(curr, new_vid_df)

In [12]:
conn.commit()